In [20]:
#!gsutil -m cp -r gs://the_impostor_hunt/data ./  

In [1]:
# ler o arquivo data/train.csv
# para cada id na tabela train_roles no formato com 4 numeros (0001, 0002, 0010)
# ler cada um dos arquivos na pasta data/train/pasta

In [2]:
import pandas as pd
import json

In [3]:
train_rules = pd.read_csv("data/train.csv")

In [4]:
train_rules.head()

,id,real_text_id
0,0,1
1,1,2
2,2,1
3,3,2
4,4,2


In [5]:
train_rules['fake_text_id'] = train_rules['real_text_id'].apply(lambda x: 2 if x == 1 else 1)

In [6]:
train_rules.head()

,id,real_text_id,fake_text_id
0,0,1,2
1,1,2,1
2,2,1,2
3,3,2,1
4,4,2,1


In [7]:
train_rules['article'] = train_rules['id'].apply(lambda x: str(x).zfill(4))

In [8]:
train_rules.head()

,id,real_text_id,fake_text_id,article
0,0,1,2,0000
1,1,2,1,0001
2,2,1,2,0002
3,3,2,1,0003
4,4,2,1,0004


In [9]:
def cria_caminho_article(text_id, article):
    caminho = f"data/train/article_{article}/file_{text_id}.txt"
    return caminho

In [10]:
train_rules['real_text_file'] = train_rules[['real_text_id', 'article']].apply(lambda x: cria_caminho_article(x['real_text_id'], x['article']), axis=1)
train_rules['fake_text_file'] = train_rules[['fake_text_id', 'article']].apply(lambda x: cria_caminho_article(x['fake_text_id'], x['article']), axis=1)

In [11]:
train_rules.head()

,id,real_text_id,fake_text_id,article,real_text_file,fake_text_file
0,0,1,2,0000,data/train/article_0000/file_1.txt,data/train/article_0000/file_2.txt
1,1,2,1,0001,data/train/article_0001/file_2.txt,data/train/article_0001/file_1.txt
2,2,1,2,0002,data/train/article_0002/file_1.txt,data/train/article_0002/file_2.txt
3,3,2,1,0003,data/train/article_0003/file_2.txt,data/train/article_0003/file_1.txt
4,4,2,1,0004,data/train/article_0004/file_2.txt,data/train/article_0004/file_1.txt


In [12]:
def leitura_arquivo(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read().strip()
    return text

In [13]:
train_rules['real_text'] = train_rules['real_text_file'].apply(leitura_arquivo)

In [14]:
train_rules['fake_text'] = train_rules['fake_text_file'].apply(leitura_arquivo)

In [15]:
train_rules

,id,real_text_id,fake_text_id,article,real_text_file,fake_text_file,real_text,fake_text
0,0,1,2,0000,data/train/article_0000/file_1.txt,data/train/article_0000/file_2.txt,The VIRSA (Visible Infrared Survey Telescope A...,The China relay network has released a signifi...
1,1,2,1,0001,data/train/article_0001/file_2.txt,data/train/article_0001/file_1.txt,The project aims to achieve an accuracy level ...,China\nThe goal of this project involves achie...
2,2,1,2,0002,data/train/article_0002/file_1.txt,data/train/article_0002/file_2.txt,Scientists can learn about how galaxies form a...,Dinosaur eggshells offer clues about what dino...
3,3,2,1,0003,data/train/article_0003/file_2.txt,data/train/article_0003/file_1.txt,The importance for understanding how stars evo...,China\nThe study suggests that multiple star s...
4,4,2,1,0004,data/train/article_0004/file_2.txt,data/train/article_0004/file_1.txt,Analyzing how fast stars rotate within a galax...,Dinosaur Rex was excited about his new toy set...
...,...,...,...,...,...,...,...,...
90,90,2,1,0090,data/train/article_0090/file_2.txt,data/train/article_0090/file_1.txt,A key focus of modern cosmology is to understa...,A main focus of modern cosmology is to underst...
91,91,1,2,0091,data/train/article_0091/file_1.txt,data/train/article_0091/file_2.txt,"APEX, as its name suggests, serves as a guide ...","APEX, as its name suggests, serves as a guide ..."
92,92,2,1,0092,data/train/article_0092/file_2.txt,data/train/article_0092/file_1.txt,FORS1 and FORS2 are early instruments of the V...,FORS1 and FORS2 are early instruments of the V...
93,93,2,1,0093,data/train/article_0093/file_2.txt,data/train/article_0093/file_1.txt,The observations of the Pluto-Charon binary an...,The observations of the Pluto-Charon system an...


In [19]:
# Criar os dois dataframes com as colunas relevantes, incluindo o artigo
df_real = train_rules[['article', 'real_text']].copy()
df_real.columns = ['article', 'texto']  # Padroniza nomes
df_real['label'] = 1

df_fake = train_rules[['article', 'fake_text']].copy()
df_fake.columns = ['article', 'texto']
df_fake['label'] = 0

# Concatenar os dois dataframes (real + fake)
df_completo = pd.concat([df_real, df_fake], ignore_index=True)

# Exibir resultado
print(df_completo.head())

  article                                              texto  label
0    0000  The VIRSA (Visible Infrared Survey Telescope A...      1
1    0001  The project aims to achieve an accuracy level ...      1
2    0002  Scientists can learn about how galaxies form a...      1
3    0003  The importance for understanding how stars evo...      1
4    0004  Analyzing how fast stars rotate within a galax...      1


In [20]:
df_completo

,article,texto,label
0,0000,The VIRSA (Visible Infrared Survey Telescope A...,1
1,0001,The project aims to achieve an accuracy level ...,1
2,0002,Scientists can learn about how galaxies form a...,1
3,0003,The importance for understanding how stars evo...,1
4,0004,Analyzing how fast stars rotate within a galax...,1
...,...,...,...
185,0090,A main focus of modern cosmology is to underst...,0
186,0091,"APEX, as its name suggests, serves as a guide ...",0
187,0092,FORS1 and FORS2 are early instruments of the V...,0
188,0093,The observations of the Pluto-Charon system an...,0


In [35]:
n_artigo = '0002'
print(df_completo[df_completo['article']==n_artigo].label.iloc[0])
df_completo[df_completo['article']==n_artigo].texto.iloc[0]

1


"Scientists can learn about how galaxies form and evolve through two methods: observing many distant galaxies or studying specific nearby ones closely enough for detailed analysis using techniques like spectroscopy (measuring light). Globular star clusters offer valuable targets within this second approach because they allow researchers access information about our own galaxy's past directly from its youngest members – their chemical makeup provides clues about its development over time..\nThis research builds upon previous studies highlighting how changes in elements within these star systems can reveal information about our galaxy's evolution - particularly during its early stages when it underwent dramatic transformations from clouds into structured structures like spiral arms or bulges; evidence suggests these processes were happening concurrently with stellar birth rates across various parts (and ages) within our galaxy.. Modern advancements like space telescopes allow astronomers

In [34]:
print(df_completo[df_completo['article']==n_artigo].label.iloc[1])
df_completo[df_completo['article']==n_artigo].texto.iloc[1]

0


'Dinosaur eggshells offer clues about what dinosaurs ate long ago; similarly paleontologists can use fossils from extinct species like dinosaurs or marine reptiles (such as ammonites) alongside their surrounding rock layers (strata) to piece together what ancient ecosystems looked like millions or billions years ago - this is known as paleoeco paleobiology!\nDinosaurs died out around 66 million years ago due perhaps some catastrophic event such as an asteroid impact which could explain why so many dinosaur fossils are only found near South America but not anywhere else on Earth!\nThe discovery has led scientists on an exciting journey through time where they hope one day maybe they will find evidence suggesting dinosaurs may have been alive during those times when giant sauropods roamed Earth!'